### 一. 同步异步与阻塞非阻塞的区分
#### 1. 同步异步: 从2个任务的执行顺序上区分  
* 同步: 指一个任务, 只有当另一个任务返回后才能继续执行本任务
* 异步: 指一个任务只是发起一个通知告诉另一个任务可以执行了, 然后就继续执行自身的任务
    
 同步可以保证2个任务的执行顺序, 而异步不能
 
 
#### 2. 阻塞和非阻塞: 从1个线程等待另一个线程返回时的状态区分
* 阻塞: 一个线程如果在等待另一个线程返回时, 自身处于挂起状态, 就是阻塞的  
* 非阻塞: 反之则是非阻塞
    
    
#### 3. 同步阻塞和同步非阻塞  
 同步表示必须保证任务顺序, 阻塞表示线程等待另一个线程时是否会被挂起, 因此:
* 同步非阻塞: 一个线程必须等待一个方法调用完毕才能继续执行接下来的任务, 且在等待过程中还可以执行其他方法, 则是同步非阻塞的
* 同步阻塞:  一个线程必须等待一个方法调用完毕才能继续执行接下来的任务, 但是等待过程中自己处于挂起状态, 则是同步阻塞的

### 二. 几个概念
#### 1. 用户空间和内核空间  
现代操作系统使用虚拟存储器([软硬件接口/5-存储器层级结构/5.7-虚拟存储器](https://nbviewer.jupyter.org/github/lj72808up/JavaFeatures/blob/master/%E8%AE%A1%E7%AE%97%E6%9C%BA%E8%BD%AF%E7%A1%AC%E4%BB%B6%E6%8E%A5%E5%8F%A3/5-%E5%AD%98%E5%82%A8%E5%99%A8%E7%9A%84%E5%B1%82%E6%AC%A1%E7%BB%93%E6%9E%84/5.7-%E8%99%9A%E6%8B%9F%E5%AD%98%E5%82%A8%E5%99%A8.ipynb)), 虚拟内存将主存划分为内核空间和用户空间


#### 2. Linux IO模型  
   用户进程在读取/写入外存上的数据时, 由于使用了虚拟存储器技术, 首先会把外存的数据映射到内存上(将虚拟地址中的页映射到物理地址的页上), 此过程发生2次拷贝:  
* 为了保证系统安全, 操作系统会首先把虚拟地址中的内容拷贝到内核空间 (外存->内存)
* 再把该内核空间的数据拷贝到用户空间(进行虚拟地址映射的物理地址)上 (内存->内存, recvFrom命令)  
    
对socket传来的数据而言, 数据先从socket拷贝到内核空间, 然后再从内核空间拷贝到用户空间


### 三. Linux的几种IO模型
#### 同步IO模型
#### 1. **同步阻塞IO**   
上面已经说明, socket的数据要传给用户进程需要两步: 
* socket复制到内核空间
* 内核空间复制到用户空间

同步阻塞IO的意思是说, 这两个步骤的数据复制时, 用户进程都是处于阻塞状态, 不能做任何处理  
 ```
 所以，blocking IO的特点就是在数据复制的两个阶段都被block了。
 ```
<img src="img/20150405_VKYH.png" width="60%">
 
 
#### 2. **同步非阻塞IO**  
   当socket被设置成非阻塞的后, 用户进程要获取socket的数据时, 第一步socket复制数据到内核空间过程中, 用户进程不会阻塞, 只会被操作系统返回一个错误码, 表示数据还未准备好. 于是用户进程要轮训询问操作系统, 内核空间的数据是否已经复制完毕, 等复制完毕后, 用户进程进入阻塞状态, 等待数据从内核空间复制到用户空间   
   ```
   所以，nonblocking IO的特点是用户进程需要不断的主动询问kernel数据好了没有。
   ```
<img src="img/20152818_DXcj.png" width="70%">
   
   
#### 3. **IO多路复用(NIO)**  
   &nbsp;&nbsp;&nbsp;&nbsp;上面说到, 同步非阻塞模式需要用户进程不断询问操作系统, 内核空间是否已经把数据从socket复制完毕. 如果这个轮训是操作系统自动完成, 且一次可以同时轮训多个socket的数据是否复制完毕就好了. 于是Linux出现了epool(pool,select的升级版)  
   &nbsp;&nbsp;&nbsp;&nbsp;每个socket连接, 在Linux系统中会被表示成一个文件描述符(fd, 文件描述符也用来表示一个进程),IO多路复用操作过程如下:  
1. 用户进程调用select系统调用后进入阻塞状态
2. 操作系统开始轮训所有socket fd, 当任何一个socket fd的数据可读/可写后(不是拷贝完/写完到内核空间), select调用就会返回  
3. 用户进程在select调用返回后变成就绪状态, 调用recvFrom调用, 将数据从内核空间拷贝到用户空间
    
 &nbsp;&nbsp;&nbsp;&nbsp;IO多路复用主要解决服务器同时存在大量连接的情况, 是单线程处理多连接的模型; 与传统的多线程处理多连接, 一方面减小了多线程的开销, 另一面避免场链接带来的长时间阻塞问题  
 
 &nbsp;&nbsp;&nbsp;&nbsp;上面提到的select和epoll主要有以下几个区别:
 1. select的文件句柄数受限制, 最多能监听1024个fd, 即同时轮训1024个连接, 而epoll的fd个数没有限制  
 2. select采用轮训, fd形成一个类似数组的结构.   
    而epoll采用队列的数据结构, 对于是否有fd可读或可写, 直接查看对应的队列是否为空即可. 这是因为epoll只会对"活跃"的socket进行操作, 每个fd上配置一个回调函数, 只有fd可读或可写时才会出发回调函数(将自己加入相应的队列中), 其他idle状态句柄则不会，在这点上，epoll实现了一个"伪"AIO
 3. epool使用mmap加速内核与用户空间的消息传递。  
   无论是select,poll还是epoll都需要内核把FD消息通知给用户空间，如何避免不必要的内存拷贝就很重要，在这点上，epoll是通过内核于用户空间mmap同一块内存实现的。
     
```
第一个阶段(socket->内核空间)可以阻塞又可以不阻塞。(根据poll,select, epoll的不同而不同)
第二个阶段(内核空间->用户空间)是阻塞的。
```
<img src="img/20164149_LD8E.png" width="70%">


#### 4. **信号驱动IO**    
信号驱动式I/O：首先我们允许Socket进行信号驱动IO,并安装一个信号处理函数，进程继续运行并不阻塞。当数据准备好时，进程会收到一个SIGIO信号，可以在信号处理函数中调用I/O操作函数处理数据。过程如下图所示：
<img src="img/21091434_DsZb.png" width="70%">


#### 异步IO模型    
#### 5. 异步非阻塞IO(AIO)    
     异步阻塞IO是说, 用户发起aio_read操作之后, 函数立刻返回, 用户进程开始做其它工作, socket数据复制到内核空间, 和内核空间复制到用户空间这两个步骤都是操作系统完成的. 两步完成后, 操作系统会给用户进程发送一个signal或执行一个基于线程的回调函数来完成这次 IO 处理过程，告诉它read操作完成了.
<img src="img/20175459_gtgw.png" width="70%">

#### 几种IO之间的区别
<img src="img/21095604_vhHX.png" width="70%">

### 四. Zero Copy
#### 1. **通过`FileChannel`的`transfer`实现内核态和用户态之间的零拷贝**    
虚拟存储技术的出现,使进程之间占用的内存不会互相干扰, 为了进一步增加安全性, 内存被划分为了内核空间和用户空间, 所有IO设备中的数据都要先复制到内核空间, 再有内核空间复制到用户空间后才能使用. 如果要在两个IO设备之间传输数据, 且其中至少有一个IO源是磁盘上的文件, 则可以使用`FileChannel`的`transferTo`或`transferFrom`, 让数据直接在内存空间上流转, 而不用经过用户空间产生多余的拷贝

1. `transferFrom()`  
如下打开两个文件channel,把数据从一个文件写入到另一个文件.
 下面的position和count用来描述源文件.
    * position: 从toChannel的哪个位置开始写
    * count: 从fromChannel中读多少数据写入 
    * 如果FileChannel的transferFrom是SocketChannel,且将socket设置为非阻塞模式, 那么当调用transferFrom()方法时, 会立刻把数据从socket的缓冲区中读取出来; 即便读取的大小没到count指定的大小, 且后续仍会有数据从socket中来, 也不能再次写入到FileChannel中
     <img src="img/transforfrom.png" width="70%">

2. `transferTo()`   
 同上述功能一样, 将数据从fromChannel拷贝到toChannel
    * position: 从fromChannel的那个位置开始读
    * count: 从fromChannel中读多少数据
    * 如果fromChannel是SocketChannel,且将socket设置为非阻塞模式,一样会存在transferFrom的问题: 立刻读取, 且后续即使有数据到来也不能继续读取
    <img src="img/transforto.png" width="70%">
        
        
#### 2. **传统的数据传输方式**  
  像这种从文件读取数据然后将数据通过网络传输给其他的程序的方式其核心操作就是如下两个调用：
```java
File.read(fileDesc,buf,len);
Socket.send(socket,buf,len);
```
其上操作看上去只有两个简单的调用，但是其内部过程却要经历四次用户态和内核态的切换以及四次的数据复制操作：
<img src="img/transfor3.png" width="60%">  
上图展示了数据从文件到socket的内部流程。   
下面看下用户态和内核态的切换过程：   
<img src="img/trans5.png" width="70%">  
步骤如下：
1. read()的调用引起了从用户态到内核态的切换（看图二），内部是通过sys_read()（或者类似的方法）发起对文件数据的读取。数据的第一次复制是通过DMA（直接内存访问）将磁盘上的数据复制到内核空间的缓冲区中；
2. 数据从内核空间的缓冲区复制到用户空间的缓冲区后，read()方法也就返回了。此时内核态又切换回用户态，现在数据也已经复制到了用户地址空间的缓存中；
3. socket的send()方法的调用又会引起用户态到内核的切换，第三次数据复制又将数据从用户空间缓冲区复制到了内核空间的缓冲区，这次数据被放在了不同于之前的内核缓冲区中，这个缓冲区与数据将要被传输到的socket关联；
4. send()系统调用返回后，就产生了第四次用户态和内核态的切换。随着DMA单独异步的将数据从内核态的缓冲区中传输到协议引擎发送到网络上，有了第四次数据复制。


#### 3. **zero copy的数据传输方式**  
zero copy主要用到的技术是: 
* DMA数据传输 
* 内存地址映射  

下图展示了通过transferTo实现数据传输的路径：
<img src="img/trans1.png" width="50%">  
下图展示了内核态、用户态的切换情况：
<img src="img/trans2.png" width="60%">  
使用transferTo()方式所经历的步骤：
1. transferTo调用会引起DMA将文件内容复制到读缓冲区(内核空间的缓冲区)，然后数据从这个缓冲区复制到另一个与socket输出相关的内核缓冲区中； 
2. 第三次数据复制就是DMA把socket关联的缓冲区中的数据复制到协议引擎上发送到网络上。  

这次改善，我们是通过将内核、用户态切换的次数从四次减少到两次，将数据的复制次数从四次减少到三次(只有一次用到cpu资源)。但这并没有达到我们零复制的目标。如果底层网络适配器支持收集操作的话，我们可以进一步减少内核对数据的复制次数。在内核为2.4或者以上版本的linux系统上，socket缓冲区描述符将被用来满足这个需求。这个方式不仅减少了内核用户态间的切换，而且也省去了那次需要cpu参与的复制过程。从用户角度来看依旧是调用transferTo()方法，但是其本质发生了变化：
1. 调用transferTo方法后数据被DMA从文件复制到了内核的一个缓冲区中；
2. 数据不再被复制到socket关联的缓冲区中了，仅仅是将一个描述符（包含了数据的位置和长度等信息）追加到socket关联的缓冲区中。DMA直接将内核中的缓冲区中的数据传输给协议引擎，消除了仅剩的一次需要cpu周期的数据复制。